In [123]:
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from bs4 import BeautifulSoup
import pandas as pd
import csv

CODE TEST PROXY

In [124]:
import requests

# Enter a proxy IP address and port.
proxy = 'http://154.6.98.26:3128'

# Initialize a URL.
url = 'https://www.malt.fr'

# Send a GET request to the url and pass the proxy as a parameter.
page = requests.get(url,
proxies={"http": proxy, "https": proxy})

# Prints the content of the requested url.
page.text

'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><link href="/cdn-cgi/styles/challenges.css" rel="stylesheet"><meta http-equiv="refresh" content="375"></head><body class="no-js"><div class="main-wrapper" role="main"><div class="main-content"><noscript><div id="challenge-error-title"><div class="h2"><span class="icon-wrapper"><div class="heading-icon warning-icon"></div></span><span id="challenge-error-text">Enable JavaScript and cookies to continue</span></div></div></noscript></div></div><script>(function(){window._cf_chl_opt={cvId: \'2\',cZone: "www.malt.fr",cType: \'managed\',cNounce: \'47770\',cRay: \'82605ae2ef5336a9\',cHash: \'a20197bbf333777\',cUPMDTk: "\\/?__cf_chl_tk=kImscxDjYyfKPCeuIHdCGxjT0DHc_kr3K4F5LtzRNaU-1699976448-0-g

INITIATE PROXY POOL

In [125]:
import requests
from itertools import cycle
import traceback

# Enter proxy ip's and ports in a list.
proxies = {
    ''
}

def rotate_proxy(proxy_pool): # Rotate the proxy pool. If the proxy is not working, it will be skipped.
    proxy = next(proxy_pool)
    while not requests.get('https://www.google.com', proxies={'http': proxy}).status_code == 200:
        proxy = next(proxy_pool)
    return proxy


proxy_pool = cycle(proxies) # Create the proxy pool.

In [126]:
def scrap_malt_user(links): # links is a list of links to scrap.
    all_data = []

    for link in links:
        print(link)
        proxy = rotate_proxy(proxy_pool)
        print(proxy)
        
        firefox_options = webdriver.FirefoxOptions()
        firefox_options.add_argument(f'--proxy-server=%s' % proxy)
        driver = webdriver.Firefox(options=firefox_options)
        
        # get origin IP address
        driver.get("http://httpbin.org/ip")
        print(driver.page_source)
        break
        
        # chrome_options = webdriver.ChromeOptions()
        # chrome_options.add_argument('--proxy-server=%s' % proxy)
        # driver = uc.Chrome(options=chrome_options, use_subprocess=True)

        wait = WebDriverWait(driver, 10)
        driver.get(link)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        data = {}

        price_element = soup.find('div', {'data-testid': 'profile-price'})
        if price_element:
            price = price_element.find('span', class_='block-list__price').text
            data['price'] = price.strip()
        
        # Récupérer l'expérience
        experience_element = soup.find('span', text='Expérience')
        if experience_element:
            experience = experience_element.find_next('span', class_='profile-indicators-content').text
            data['experience'] = experience.strip()
        
        # Récupérer le taux de réponse
        response_rate_element = soup.find('span', text='Taux de réponse')
        if response_rate_element:
            response_rate = response_rate_element.find_next('span', class_='profile-indicators-content').text
            data['response_rate'] = response_rate.strip()
        
        # Récupérer le temps de réponse
        response_time_element = soup.find('span', text='Temps de réponse')
        if response_time_element:
            response_time = response_time_element.find_next('span', class_='profile-indicators-content').text
            data['response_time'] = response_time.strip()
            
        # Récupérer le nom 
        name_element = soup.find('div', {'data-testid': 'profile-fullname'})
        if name_element:
            name = name_element.text
            data['name'] = name.strip()
            
        # Récupérer le métier
        headline_element = soup.find('div', {'data-testid': 'profile-headline'})
        if headline_element:
            headline = headline_element.text
            data['headline'] = headline.strip()
            
        # Récupérer le nombre de missions
        missions_element = soup.find('div', {'data-testid': 'profile-counter-missions'})
        if missions_element:
            missions = missions_element.find('strong').text
            data['missions'] = missions.strip()
            
        # Récupérer toutes les catégories
        categories_elements = soup.find_all('li', {'class': 'categories__list-item'})
        categories = [category.find('a').text for category in categories_elements]
        data['categories'] = categories
        
        
        # Récupérer les compétences
        competences_element = soup.find_all('div', {'class': 'profile-expertises__content-list-item__label'})
        competences = [competence.find('a', class_='joy-link joy-link_teal').text.strip() for competence in competences_element]

        data['competences'] = competences
        
        # Récupérer le statut "Supermalter"
        supermalter_element = soup.find('span', class_='joy-badge-level__tag blue')
        if supermalter_element:
            supermalter = supermalter_element.get_text(strip=True)
            data['supermalter'] = supermalter
            
        # Récupérer la localisation
        location_element = soup.find('dl', {'class': 'profile__location-and-workplace-preferences__item'})
        if location_element:
            location_label = location_element.find('dt', {'data-testid': 'profile-location-address-label'})
            location_value = location_element.find('dd', {'data-testid': 'profile-location-preference-address'})

            if location_label and location_value:
                location = {location_label.text: location_value.text}
                data['location'] = location
                
        # Récupérer la préférence de télétravail
        teletravail_element = soup.find('dl', {'class': 'profile-page-mission-preferences__item'})
        if teletravail_element:
            teletravail_label = teletravail_element.find('dt')
            teletravail_value = teletravail_element.find('dd')

            if teletravail_label and teletravail_value:
                teletravail_preference = {teletravail_label.text: teletravail_value.text}
                data['teletravail_preference'] = teletravail_preference
                
        # Récupérer le nombre de recommandations
        recommendations_element = soup.find('span', {'data-testid': 'profile-counter-recommendations'})
        if recommendations_element:
            recommendations_count = int(recommendations_element.text.split()[0])
            data['recommendations'] = recommendations_count   
            

        # Récupérer le message de présentation
        presentation_element = soup.find('div', {'class': 'profile-description__content'})
        if presentation_element:
            presentation_message = presentation_element.get_text(strip=True)
            data['presentation'] = presentation_message
            
            
        all_data.append(data)
        
        driver.quit()

    return all_data

In [127]:
profile_links = pd.read_csv('../data/links.csv')
profile_links['profil'] = profile_links['profil'].apply(lambda x: x.replace('https://www.malt.fr/profile/', ''))
profile_links = profile_links['profil'].tolist()
profile_links = [f'https://www.malt.fr/profile/{link}' for link in profile_links]

data = scrap_malt_user(profile_links[:3])
data

https://www.malt.fr/profile/bricetillet
38.62.222.144:3128
<html platform="linux" class="theme-dark" dir="ltr"><head><meta http-equiv="Content-Security-Policy" content="default-src 'none' ; script-src resource:; "><link rel="stylesheet" type="text/css" href="chrome://devtools-jsonview-styles/content/main.css"><script type="text/javascript" charset="utf-8" async="" data-requirecontext="_" data-requiremodule="viewer-config" src="resource://devtools/client/jsonview/viewer-config.js"></script><script type="text/javascript" charset="utf-8" async="" data-requirecontext="_" data-requiremodule="json-viewer" src="resource://devtools/client/jsonview/json-viewer.js"></script><script type="text/javascript" charset="utf-8" async="" data-requirecontext="_" data-requiremodule="devtools/client/shared/vendor/react-dom" src="resource://devtools/client/shared/vendor/react-dom.js"></script><script type="text/javascript" charset="utf-8" async="" data-requirecontext="_" data-requiremodule="devtools/client/s

[]

In [128]:
def save_to_csv(data, csv_filename):
    # Assurez-vous que la liste de données n'est pas vide
    if not data:
        print("Aucune donnée à enregistrer.")
        return

    # Créez une liste de noms de colonnes pour le CSV en incluant les nouveaux éléments
    fieldnames = ['name', 'headline', 'price', 'experience', 'response_rate', 'response_time', 'missions', 'categories', 'competences', 'supermalter', 'location','presentation', 'recommendations', 'teletravail_preference']

    # Ouvrir le fichier CSV en mode écriture
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Écrire les en-têtes
        writer.writeheader()

        # Écrire les données
        for entry in data:
            writer.writerow(entry)

    print(f"Données enregistrées dans {csv_filename}")

In [129]:
csv_filename = 'malt_data.csv'
save_to_csv(data, csv_filename)

Aucune donnée à enregistrer.
